In [0]:
!pip install pydub

import pandas as pd
import os
import librosa
import librosa.display
from PIL import Image
import shutil
from pydub import AudioSegment
import matplotlib.pyplot as plt
import cv2
# import librosa.display
from glob import glob
import numpy as np
import csv
# from skimage.feature  import hog, local_binary_pattern
import scipy.io.wavfile 
from six.moves import cPickle as pickle
from math import sqrt
from numpy import split
from google.colab.patches import cv2_imshow
from numpy import array
from keras.utils.np_utils import to_categorical
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
import tensorflow
from keras.applications import vgg16 as vgg_model
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense
from keras.initializers import RandomNormal
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Flatten
from keras.layers import LSTM, Dropout
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import RepeatVector
from keras.optimizers import SGD, Adam
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
from keras.layers import LSTM
from keras.utils.io_utils import HDF5Matrix
import h5py
# from models.my_callbacks import *
import time

# import tensorflow as tf
# tf.test.gpu_device_name()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
## TRAIN DATA FROM PICKLE FILE

data_file = '/content/gdrive/My Drive/contrast4_no_segments_train.pickle' # redefined varaible in case you have completed above steps previously.
print('Tring to load pickle from %s' % data_file)

with open(data_file, 'rb') as file:
  
    svhn_datasets = pickle.load(file)
    train_dataset_contrast = svhn_datasets['train_dataset']
    del svhn_datasets
    
print('Pickle Loaded Successfully!')

## TEST DATA FROM PICKLE FILE

data_file = '/content/gdrive/My Drive/contrast4_no_segments_test.pickle' # redefined varaible in case you have completed above steps previously.
print('Tring to load pickle from %s' % data_file)

with open(data_file, 'rb') as file:
  
    svhn_datasets = pickle.load(file)
    test_dataset_contrast = svhn_datasets['test_dataset']
    del svhn_datasets
    
print('Pickle Loaded Successfully!')



In [0]:
def split_dataset(data, labels):
    
    x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=0)
    return x_train, x_test, y_train, y_test

  
# def build_model(dropout, optimizer,lr):   # [FOR GRID SEARCH]
 
# train the model
def build_model(x_train, y_train, x_test, y_test, dropout):
    
#     print("------>> " + str(dropout) + "," + str(lr) + "," + str(optimizer) + " <<------")
    
    # define parameters
    num_rows = x_train.shape[1]
    num_columns = 775
    num_channels = 1

    num_labels = 10
    filter_size = 2

    # Construct model 
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
#     model.add(Dropout(0.2))

    model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
#     model.add(Dropout(0.2))

    model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
#     model.add(Dropout(0.2))

    model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
#     model.add(Dropout(0.2))
#     model.add(GlobalAveragePooling2D())

    model.add(Dropout(dropout))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))

    model.add(Dense(num_labels, activation='softmax'))
    
    print(model.summary())
    
    return model
  

def acc(y_true, y_pred):
    return np.equal(np.argmax(y_true, axis=-1), np.argmax(y_pred, axis=-1)).mean()

In [0]:
x_train_mfcc = train_dataset_mfcc['X']
y_train_mfcc = train_dataset_mfcc['y']
x_test_mfcc = test_dataset_mfcc['X']
y_test_mfcc = test_dataset_mfcc['y']

x_train_centroid = train_dataset_centroid['X']
y_train_centroid = train_dataset_centroid['y']
x_test_centroid = test_dataset_centroid['X']
y_test_centroid = test_dataset_centroid['y']

x_train_cens = train_dataset_cens['X']
y_train_cens = train_dataset_cens['y']
x_test_cens = test_dataset_cens['X']
y_test_cens = test_dataset_cens['y']

x_train_contrast = train_dataset_contrast['X']
y_train_contrast = train_dataset_contrast['y']
x_test_contrast = test_dataset_contrast['X']
y_test_contrast = test_dataset_contrast['y']

del train_dataset_mfcc
del train_dataset_centroid
del train_dataset_contrast
del train_dataset_cens

print(x_train_mfcc.shape)
print(y_train_mfcc.shape)
print(x_test_mfcc.shape)
print(y_test_mfcc.shape)
print('\n\n')

print(x_train_cens.shape)
print(y_train_cens.shape)
print(x_test_cens.shape)
print(y_test_cens.shape)
print('\n\n')

print(x_train_centroid.shape)
print(y_train_centroid.shape)
print(x_test_centroid.shape)
print(y_test_centroid.shape)
print('\n\n')

print(x_train_contrast.shape)
print(y_train_contrast.shape)
print(x_test_contrast.shape)
print(y_test_contrast.shape)


In [0]:
x_train = np.concatenate((x_train_mfcc, x_train_cens, x_train_centroid, x_train_contrast), axis=1)
x_test = np.concatenate((x_test_mfcc, x_test_cens, x_test_centroid, x_test_contrast), axis=1)
y_train = y_train_mfcc
y_test = y_test_mfcc

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [0]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 775, 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 775, 1)

print(x_train.shape)
print(x_test.shape)

In [0]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=0)

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

print(x_val.shape)
print(y_val.shape)


In [0]:
dropout = 0.3
model = build_model(x_train, y_train, x_test, y_test, dropout)

In [0]:
from keras.optimizers import Nadam
optimizer = Nadam(lr=0.0005,
                  beta_1=0.9,
                  beta_2=0.999,
                  epsilon=1e-08,
                  schedule_decay=0.004)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"]) 

In [0]:
verbose, epochs, batch_size = 1, 20, 32
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data= (x_test, y_test))


In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
fig1 = plt.gcf()
plt.show()
# fig1.savefig('model_accuracy_ECN2.png', dpi=100)

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
fig2 = plt.gcf()
plt.show()
# fig2.savefig('model_loss_ECN2.png', dpi=100)

In [0]:
from keras.models import load_model
# model.save('my_model.h5')  

model = load_model('/content/gdrive/My Drive/my_model.h5')

In [0]:
print('⦁	MECN	⦁\n\n')

score = model.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy All: ", score[1])
print('\n')

for i in range(10):
  score = model.evaluate(x_test[np.where(y_test==i)], y_test[np.where(y_test==i)], verbose=0)
  print('Testing Accuracy_' + str(i) + ': ', score[1])

